In [3]:
video_url = "https://youtu.be/gZQgOWPt5hg?si=X6WABd7jMlVfZwal"

In [19]:
# GPU 0 으로 강제 설정
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#device = torch.device("cuda:" if torch.cuda.is_available() else "cpu")

# 기능.1 : 김형섭

In [20]:
import os
import yt_dlp
from datetime import datetime
# from fastapi import HTTPException

OUTPUT_FOLDER = "/home/wanted-1/potenup-workspace/Project/project3/team2/SY/평가"

In [21]:
def download_audio(video_url: str):
    """유튜브 비디오에서 오디오 추출하여 output_folder에 저장"""

    os.makedirs(OUTPUT_FOLDER, exist_ok=True)       # output_folder 생성

    # 타임스탬프 추가 (파일명 중복 방지)
    # timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    audio_filename = "extracted_audio"
    audio_path = os.path.join(OUTPUT_FOLDER, audio_filename)

    # 유튜브 오디오 다운로드 옵션 설정
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": audio_path,                      # 저장될 경로 지정
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192"
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])               # 유튜브에서 오디오 다운로드
    except Exception as e:
        print(e)
        # raise HTTPException(status_code=500, detail=f"오디오 추출 실패: {str(e)}")

    return audio_path                               # 다운로드된 파일 경로 반환

In [22]:
audio_path = download_audio(video_url)

[youtube] Extracting URL: https://youtu.be/gZQgOWPt5hg?si=X6WABd7jMlVfZwal
[youtube] gZQgOWPt5hg: Downloading webpage


[youtube] gZQgOWPt5hg: Downloading tv client config
[youtube] gZQgOWPt5hg: Downloading player b191cf34
[youtube] gZQgOWPt5hg: Downloading tv player API JSON
[youtube] gZQgOWPt5hg: Downloading ios player API JSON
[youtube] gZQgOWPt5hg: Downloading m3u8 information
[info] gZQgOWPt5hg: Downloading 1 format(s): 251
[download] Destination: /home/wanted-1/potenup-workspace/Project/project3/team2/SY/평가/extracted_audio
[download] 100% of    4.36MiB in 00:00:00 at 45.36MiB/s    
[ExtractAudio] Destination: /home/wanted-1/potenup-workspace/Project/project3/team2/SY/평가/extracted_audio.mp3
Deleting original file /home/wanted-1/potenup-workspace/Project/project3/team2/SY/평가/extracted_audio (pass -k to keep)


In [23]:
audio_path

'/home/wanted-1/potenup-workspace/Project/project3/team2/SY/평가/extracted_audio'

# 기능.2 : 지서연

In [24]:
import torch
import gc
import os

def initialize_gpu():
    """
    GPU 캐시, IPC, 가비지 컬렉션 실행 후 GPU 동기화 및 초기화
    """
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()
    torch.cuda.synchronize()
    # os.system("nvidia-smi --gpu-reset")  # 필요시 주석 해제 (관리자 권한 필요)


In [25]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
device = "cuda:0" if torch.cuda.is_available() else "cpu"

def load_asr_pipeline(model_id: str, torch_dtype,  device: str):
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id,
        torch_dtype=torch_dtype,
        low_cpu_mem_usage=True,
        use_safetensors=True
    )
    model.to(device)
    processor = AutoProcessor.from_pretrained(model_id)
    
    asr_pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype, #torch.float16,
        device=device,
    )
    return asr_pipe


In [28]:
#device = "cuda:1" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 

model_id = "openai/whisper-large"

load_asr_pipeline(model_id, torch_dtype, device)

Device set to use cuda:0


In [29]:
import re
from collections import defaultdict

def remove_extra_repeated_words(text: str) -> str:
    """
    동일한 한글 단어가 3번 이상 반복될 경우 2개만 남기고 나머지 삭제
    (숫자 및 특수문자는 유지)
    """
    word_count = defaultdict(int)
    word_positions = {}
    
    for match in re.finditer(r"[가-힣]+", text):
        word = match.group()
        word_count[word] += 1
        word_positions.setdefault(word, []).append(match.start())
    
    result = list(text)
    for word, positions in word_positions.items():
        if word_count[word] > 2:
            for i in range(2, len(positions)):
                start_idx = positions[i]
                for j in range(len(word)):
                    result[start_idx + j] = ""
    return "".join(result)


In [30]:
#!/usr/bin/env python3
import os
# from gpu_utils import initialize_gpu
# from asr_model import load_asr_pipeline
# from text_utils import remove_extra_repeated_words

def main():
    # GPU 초기화
    initialize_gpu()
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 

    model_id = "openai/whisper-small"
    asr_pipe = load_asr_pipeline(model_id, torch_dtype, device)

    # 음성 인식 설정 (forced_decoder_ids를 생략하여 내부 language 옵션 사용)
    generate_kwargs = {
        "max_new_tokens": 445,
        "num_beams": 1,
        "condition_on_prev_tokens": False,
        "compression_ratio_threshold": 1.35,
        "temperature": (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
        "logprob_threshold": -1.0,
        "no_speech_threshold": 0.6,
        "return_timestamps": True,
        # language 옵션은 내부적으로 처리하도록 합니다.
    }
    
    # 오디오 파일 경로
    audio_path = r'/home/wanted-1/potenup-workspace/Project/project3/team2/SY/평가/extracted_audio.mp3'
    # language 옵션을 generate_kwargs에 포함하지 않고, 파이프라인 내부에서 설정하게 함.
    result = asr_pipe(audio_path, generate_kwargs={"language": "korean"}, return_timestamps=True)
    
    test_text = result.get("text", "")
    cleaned_text = remove_extra_repeated_words(test_text)
    print("처리된 텍스트:", cleaned_text)
    
    output_file = "/home/wanted-1/potenup-workspace/Project/project3/team2/SY/평가/processed_text.txt"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(cleaned_text)
    print(f"최종 텍스트가 '{output_file}'에 저장되었습니다.")

  


In [31]:
main()

Device set to use cuda:0
/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=korean, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=korean.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


처리된 텍스트:  요즘 은행 대출 전구 가보신 분들은 대출 밖에 얼마나 힘든지 세삼 느끼실 겁니다. 하반기에는 더 어려워질 거라는데 왜 그런 건지 어떻게 하면  잘 받을 수 있을지 상담 받아보겠습니다. 레오대출 연구소 김은진 대표와 함께하겠습니다. 대표님 어서 오세요. 안녕하세요. 요즘은  받으려면 계절 잘 타야 된다고요. 이제 강을 이후 되면  창고가 다쳐버려서 올해도 그럴까요. 네 맞습니다. 이제 거의 공식화 된 것 같습니다. 그래서 올해도 역시 하반기에는 대출의 문턱이 더 높아질 것으로 보이고 있습니다. 여러 가지가 바뀔 예정이긴 한데요. 특히 눈에 띄는 것이 전세대출입니다. 그동안 전세대출은 규제가 조금 되지 않았던 부분이긴 한데 전세대출의 보증기관이 저기 보시는 대로 주택금융공사, 주택 보시 보증공사, 서울 보증 이렇게 세 군대가 있는데요. 여기서 보증해주면 은행이 대출을 해준다. 가운데 항상 등판을 해야 하는 세 곳의 기관인데  그 보증 비율을 축소하겠다고 발표를 했습니다.  비율을 100%에서 90%로 줄인다? 이러면  받는 데 어떤 영향이 있는지?   100%라고 하면 은행에서는 그냥 100% 다 받을 수 있기 때문에 전혀 손해가 없죠. 그런데 90%만, 혹시 부실이 나면 90%만 물어주겠다 이런 의미이기 때문에 은행 입장에서는 리스크가 발생합니다. 10%의 돈이 받지 못할 수도 있기  그래서 은행에서는 심사를 조금  강화할 수밖에 없는 상황이고요. 그리고 리스크가 있으니까 혹시  금리에도 영향을 미칠 수가 있는 그런 상황이 될  있습니다. 그러다 보면 저희가 서민들이 많이 거주하는 곳이 빌라라든지 다세대 열림 이런 곳이 많은데  부분에서도 역시 대출이 어려 워낙 할  있고요. 그러다 보면 서민들이 점점 월세로 가야 할 수도 있는 상황이 발생할  . 7월부터는 3단계 스트레스 DSR, 이거 도입된다면서요? 네, 뭐예요? 스트레스 DSR, 항상  이슈가 되는  단어인데요.  DSR이라는 것은 소득 대비,  한도가 정해지는  규정이라고 보시면 되는데

## 3. Gemma-2-2b-it

In [32]:
detailed_prompt = """
After reading the original text below, please perform the following steps in order:

1. Translate the original text into English.
2. Summarize the translated text by extracting only the most important core points. Present each key point as a numbered item. Each numbered item must:
   - Begin with the item number followed by a period and a space (e.g., "1. ").
   - Contain only the key point in a simple and direct sentence.
   - End with a period.
3. Finally, translate the numbered summary into Korean.

Important:
- The final output must be only the summary in Korean.
- The final summary must strictly follow the format below, with one key point per line:

   1. [First key point in Korean].
   2. [Second key point in Korean].
   3. [Third key point in Korean].
   (and so on...)

- Do not include any extra commentary, notes, bullet symbols (such as "*" or "-"), or any text like "Note:".
- Avoid repetition or duplication of words.
- Use simple and direct language.

Original Text:
{original_text}

Summary:
"""

In [33]:
# 프롬프트 
detailed_prompt = """
After reading the original text below, please perform the following steps in order:

1. Translate the original text into English.
2. Summarize the translated text by extracting only the most important core points. Present each key point as a numbered item (for example, "1.", "2.", "3.", etc.). Write each point concisely and clearly.
3. Finally, translate the numbered summary into Korean.

Important:
- The final output must be only the summary in Korean.
- Use simple and direct language.
- Do not include any extra commentary, notes, or repeated information.
- Each numbered point must be on a separate line and end with a period.
- Do not include any bullet symbols (such as "*" or "-").
- Do not write '**Note:**'
- Do not include any notes, disclaimers, or additional comments. Only output the summary.
- Do not write any 'Note:'
- Avoid duplication of word

Original Text:
{original_text}

Summary:
"""


In [34]:
import torch
import os
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, prepare_model_for_kbit_training, LoraConfig, get_peft_model

# GPU 0 으로 강제 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# 모델 불러오기 
BASE_MODEL = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map={"": torch.cuda.current_device()}
)
base_model = prepare_model_for_kbit_training(base_model)

model = PeftModel.from_pretrained(base_model, "/home/wanted-1/potenup-workspace/Project/project3/team2/SY/Gemma/path/to/save/lora_adapters_ver1_law")
model.eval()  

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 하이퍼파라미터 조정
generation_kwargs = {
    "max_length": 5000,           # 최대 생성 토큰 수 (요약 길이 제한)
    "min_length": 512,            # 최소 생성 토큰 수
    "do_sample": True,           # 샘플링 방식 사용
    "temperature": 0.3,          # 낮은 값은 결정론적, 높은 값은 다양성 증가
    "top_k": 50,                 # 상위 k개 단어 내에서 샘플링
    "top_p": 0.95,               # 누적 확률 p 내 단어에서 샘플
    "repetition_penalty" : 1.9,
}

# 텍스트 파일 경로 수정
txt_file_path = "/home/wanted-1/potenup-workspace/Project/project3/team2/SY/평가/processed_text.txt"  # 파일 경로를 수정하세요.
with open(txt_file_path, "r", encoding="utf-8") as f:
    original_text = f.read()


# 프롬프트 
detailed_prompt = """
Write the summary in complete sentences, ensuring that each sentence ends with a period.
The summary must be composed of at least 3 sentences and no more than 5 sentences.
Avoid unnecessary elaboration or repetition; be concise and clear.
Accurately reflect the core information of the original text.

### Original Text:
{original_text}

### Summary:

"""

# 프롬프트 구성
prompt = detailed_prompt.format(original_text=original_text)

#  요약 생성
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    output_tokens = model.generate(**inputs, **generation_kwargs)

# 후처리 
def post_process(text):
    # 앞뒤 공백 제거
    text = text.strip()
    # 불필요한 공백, 줄바꿈, 중복 문장 부호 제거 (예시: 느낌표, 공백 등)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'!{2,}', '!', text)
    return text

generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
# 4. "Summary:" 이후의 텍스트만 추출하여 순수 요약문만 저장
if "Summary:" in generated_text:
    summary_only = generated_text.split("Summary:")[-1].strip()
else:
    summary_only = generated_text.strip()

# "Note:"가 있다면, 그 뒤의 모든 텍스트를 제거
if "Note:" in summary_only:
    summary_only = summary_only.split("Note:")[0].strip()


# .txt 파일로 저장 
with open("/home/wanted-1/potenup-workspace/Project/project3/team2/SY/평가/generated_summary.txt", "w", encoding="utf-8") as f:
    f.write(summary_only)

print("Summary Only:", summary_only)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.29it/s]


Summary Only: 김은진과 동료들의 의견을 종합하여 볼 때, 만약 현행 장애물 및 재산권을 이용하지 않는 사람들에게 과도하게 경제적 불평등이 생길 가능성이 존재한다는 사실은 이미 증명되어 있으며 따라서, 위 법률에 근거해서 실제적으로 감액시킬 필요가 있다. 또한, 이러한 문제는 사회 전체적인 환경이나 개인의 취지를 고려해야하기 때문에 각자가 자신의 권리와 책임을 충족하려고 노력할 것이다. 따라서, 김은진에게 주어진 업무의 목적을 달성함 위한 직업을 선택할 자격은 없다.


### 성능 평가

In [ ]:
import torch
import math
from transformers import AutoTokenizer, AutoModelForCausalLM
from nltk.translate.meteor_score import meteor_score

def compute_perplexity(text, model, tokenizer):
    """
    주어진 텍스트에 대한 Perplexity(PPL)를 계산하는 함수.
    텍스트를 토크나이즈한 후, 모델의 loss를 계산하고, 이를 기반으로 perplexity를 구함.
    """
    # 텍스트 토크나이즈 및 텐서 변환
    encodings = tokenizer(text, return_tensors="pt")
    input_ids = encodings.input_ids.to(model.device)
    
    # 모델 출력 (라벨에 input_ids를 넣어 self-supervised loss 계산)
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
    
    # Perplexity 계산 (exp(loss))
    ppl = math.exp(loss.item())
    return ppl


if __name__ == "__main__":
    # 평가에 사용할 모델과 토크나이저 (예시로 GPT-2 사용)
    model_name = "gpt2"  # 필요한 모델 이름으로 변경하세요.
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # ---- Perplexity 평가 ----
    # 평가할 텍스트 (예시)
    text_input = "김은진과 동료들의 의견을 종합하여 볼 때, 만약 현행 장애물 및 재산권을 이용하지 않는 사람들에게 과도하게 경제적 불평등이 생길 가능성이 존재한다는 사실은 이미 증명되어 있으며 따라서, 위 법률에 근거해서 실제적으로 감액시킬 필요가 있다. 또한, 이러한 문제는 사회 전체적인 환경이나 개인의 취지를 고려해야하기 때문에 각자가 자신의 권리와 책임을 충족하려고 노력할 것이다. 따라서, 김은진에게 주어진 업무의 목적을 달성함 위한 직업을 선택할 자격은 없다"
    ppl = compute_perplexity(text_input, model, tokenizer)
    print("Perplexity (PPL):", ppl)



Perplexity (PPL): 7.83115305459753


: 